<a href="#1.-K-fold" style="margin-left: 0px;">1. K-fold</a>   
<a href="#2.-Групповой-K-fold" style="margin-left: 0px;">2. Групповой K-fold</a>  
<a href="#3.-StratifiedKFold" style="margin-left: 0px;">3. StratifiedKFold</a>  
<a href="#4.-K-fold-time-series" style="margin-left: 0px;">4. K fold time series</a>

1. K-fold - просто делим на равные части  
2. Групповой K-fold - разные группы сбалансированно раскиданы по разным фолдам не пересекаясь  
3. StratifiedKFold - каждый класс определенного признака в равной пропорции присутствует в каждом фолде  
4. K fold time series - каждый фолд не пересекается с друим по времени (время это порядок строки, не смотрим на конкретный столбец)

In [1]:
import numpy as np
import pandas as pd

from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
import random
import itertools
import math

In [2]:
# Фиксируем случайные сиды
seed = 42
np.random.seed(seed)
random.seed(seed)

In [3]:
data = pd.read_csv(r"../00 Data/titanic.csv")

print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Замена пропусков в Age медианным значением
data['Age'] = data['Age'].fillna(data['Age'].median())

# Замена пропусков в Embarked модой
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# Семейный размер
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

# Одиночка (1, если FamilySize = 1)
data['IsAlone'] = (data['FamilySize'] == 1).astype(int)

# Encoder
encoder = TargetEncoder(cols=['Sex', 'Embarked', 'FamilySize'])

data[['Sex', 'Embarked', 'FamilySize']] = encoder.fit_transform(
    data[['Sex', 'Embarked', 'FamilySize']], 
    data['Survived']  # Целевая переменная
)

In [5]:
# StandardScaler для всех признаков
# features = data.columns.drop('Survived')
# data[features] = StandardScaler().fit_transform(data[features])

In [6]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,0,3,0.188908,22.0,1,0,7.2500,0.339009,0.552795,0
1,1,1,0.742038,38.0,1,0,71.2833,0.553571,0.552795,0
2,1,3,0.742038,26.0,0,0,7.9250,0.339009,0.303538,1
3,1,1,0.742038,35.0,1,0,53.1000,0.339009,0.552795,0
4,0,3,0.188908,35.0,0,0,8.0500,0.339009,0.303538,1


### 1. K-fold

**входные параметры:**  
k - количество фолдов  
data - выборка

**выходные данные**:  
Кортеж из двух списков:

список индексов обучающей выборки для каждого фолда,

список индексов тестовой выборки для каждого фолда.

**Опиание:**  
реализует k-fold кросс-валидацию вручную, разбивая данные на k блоков (folds) и возвращая индексы обучающей и тестовой выборок для каждого фолда.

In [7]:
def k_fold(data,k):
    
    random.seed(42)
    
    train_indices_list = []
    test_indices_list = []
    
    size = len(data)
    
    if k>size:
        k=size
    if k<1:
        k=1
    if isinstance(k,int)==False:
        k=int(k)
    
    # Генерируем случайные индексы и перемешиваем их
    indices = list(range(size))
    random.shuffle(indices)
    
    data = data.iloc[indices]
    step = math.ceil(len(data)/k)
    
    for i in range(k):
        test_indices = indices[i * step: (i + 1) * step]  # Индексы тестового фолда
        train_indices = indices[:i * step] + indices[(i + 1) * step:]  # Остальное - тренировка
        train_indices_list.append(train_indices)
        test_indices_list.append(test_indices)
      
    return tuple(map(list,zip(train_indices_list,test_indices_list)))

In [8]:
# My 
indices = k_fold(data, 3)
for train, test in indices:
    print(len(train), len(test))

594 297
594 297
594 297


In [9]:
# sklearn 
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
for train, test in kf.split(data):
    print(len(train), len(test))

594 297
594 297
594 297


### 2. Групповой K-fold

**входные параметры:**  

data: выборка в виде pandas DataFrame, содержащая данные для разбиения.

group_field: имя столбца, по которому будут группироваться данные перед разбиением (например, идентификатор группы или пользователя).

k: количество фолдов для кросс-валидации.

**выходные данные:**  

Кортеж из двух списков:

список индексов обучающей выборки для каждого фолда,

список индексов тестовой выборки для каждого фолда.

**Описание:**  

1) Делит данные на k фолдов, сохраняя целостность групп (одна и та же группа не попадает одновременно в обучение и тест). 
2) Старается сохранить баланс, если всего 2 фолда и в признаке из 3 уникальных значения, то старается поделить поровну   
3) Возвращает индексы строк обучающей и тестовой выборок для каждого фолда.

❗❗❗ **Таким образом, когда мы идём по циклу, мы можем перебрать k различных разбиений**

In [10]:
def k_fold_group(data, group_field, k):
    
    random.seed(42)
      
    train_indices_list = []
    test_indices_list = []
    
    # Получаем уникальные значения в группе и их количество
    group_counts = data[group_field].value_counts().reset_index()
    group_counts.columns = [group_field, "count"]
    group_counts = group_counts.sort_values("count", ascending=False)
    unique_groups = group_counts[group_field].tolist()
    n_unique = len(unique_groups)
    
    if k>n_unique:
        k=n_unique
    
    # Создание всех возможных комбинаций групп для тренировочных и тестовых выборок
    all_combinations = list(itertools.combinations(unique_groups, k-1))
    
    combinations = []
    
    # Генерируем все возможные комбинации для тренировочных выборок
    for i in range(1, len(unique_groups) + 1):
        train_combinations = list(itertools.combinations(unique_groups, i))
        for train in train_combinations:
            # Тестовая выборка - оставшиеся группы
            test = [group for group in unique_groups if group not in train]
            combinations.append((list(train), test))

    # Преобразуем в DataFrame для удобства
    df_combinations = pd.DataFrame(combinations, columns=["Train", "Test"])

    # Добавляем количество строк для каждой комбинации Train и Test
    df_combinations['Train Count'] = [
        len(data[data[group_field].isin(train)]) for train in df_combinations['Train']
    ]
    df_combinations['Test Count'] = [
        len(data[data[group_field].isin(test)]) for test in df_combinations['Test']
    ]
    
    # Вычисляем разницу между Train Count и Test Count, берем модуль этой разницы
    df_combinations['Difference'] = (df_combinations['Train Count'] - df_combinations['Test Count']).abs()

    # Сортируем по столбцу Difference (по возрастанию)
    df_combinations = df_combinations.sort_values(by='Difference', ascending=True)

    # Берем только первые k строк
    df_combinations = df_combinations.head(k)
    
    
    # Выводим таблицу
    # print(df_combinations)
    
    if k==n_unique:
        for group in unique_groups:
            
            train_indices_list.append(data[data[group_field]!=group].index)
            test_indices_list.append(data[data[group_field]==group].index)
    
    else:
        # Собираем индексы для тренировочных и тестовых выборок
        for train, test in zip(df_combinations['Train'], df_combinations['Test']):
            # Получаем индексы для train и test выборок
            train_indices = data[data[group_field].isin(train)].index
            test_indices = data[data[group_field].isin(test)].index

            # Добавляем в списки
            train_indices_list.append(train_indices)
            test_indices_list.append(test_indices)
    
    return tuple(map(list, zip(train_indices_list, test_indices_list)))

In [11]:
data["Parch"].value_counts()

Parch
0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: count, dtype: int64

In [12]:
# My 

my_fold_indices = []
indices_kfg_my = k_fold_group(data, "Parch", 2)
for train, test in indices_kfg_my:
    print(len(train), len(test))
    my_fold_indices.append((train, test))

678 213
213 678


In [13]:
# если n_splits больше чем уникальных значений в признаке, то ошибку выдаёт
from sklearn.model_selection import GroupKFold

fold_indices = []
gkf = GroupKFold(n_splits=2)

for train, test in gkf.split(data, groups=data.Parch):
    print(len(train), len(test))
    fold_indices.append((train, test))

213 678
678 213


In [14]:
print(data.iloc[fold_indices[0][0]]['Parch'].value_counts())
print(data.iloc[fold_indices[0][1]]['Parch'].value_counts())
print('\n')

print(data.iloc[fold_indices[1][0]]['Parch'].value_counts())
print(data.iloc[fold_indices[1][1]]['Parch'].value_counts())
print('\n')

Parch
1    118
2     80
5      5
3      5
4      4
6      1
Name: count, dtype: int64
Parch
0    678
Name: count, dtype: int64


Parch
0    678
Name: count, dtype: int64
Parch
1    118
2     80
5      5
3      5
4      4
6      1
Name: count, dtype: int64




### 3. StratifiedKFold

**входные параметры:**  

data: выборка в виде pandas DataFrame, содержащая данные для разбиения.

group_field: имя столбца, по которому будут группироваться данные перед разбиением (например, идентификатор группы или пользователя).

k: количество фолдов для кросс-валидации.

**выходные данные:**  

Кортеж из двух списков:

список индексов обучающей выборки для каждого фолда,

список индексов тестовой выборки для каждого фолда.

**Описание:**  

1) Делит данные на k фолдов, сохраняя пропорции значений в group_field (например, классы), при этом одна и та же группа не попадет одновременно в тренировочную и тестовую выборку.

2) В случае, если количество объектов в группе не делится нацело на количество фолдов, остаток распределяется по первым фолдам, чтобы сделать их размер равномерным.

3) Возвращает индексы строк обучающих и тестовых выборок для каждого фолда.

In [15]:
def k_fold_stratified(data, group_field, k):
    
    if k < 2:
        raise ValueError("k должно быть больше или равно 2.")
    
    # Создаём пустые списки для хранения индексов
    train_indices_list = [[] for _ in range(k)]
    test_indices_list = [[] for _ in range(k)]
    
    # Получаем уникальные классы в group_field
    unique_groups = data[group_field].unique()
    
    # цикл для каждой группы
    for group in unique_groups:
        # Получаем все строки для текущей группы
        group_data = data[data[group_field] == group]
        group_indices = group_data.index.tolist()
        
        # Определяем индексы для разбиения
        fold_sizes = [len(group_data) // k] * k # если 11 элементов и k = 4, то fold_sizes = [2, 2, 2, 2]
        for i in range(len(group_data) % k):
            
            # Если len(group_data) = 11 и k = 4, то остаток равен 11 % 4 = 3
            # В цикле мы берём первые len(group_data) % k фолдов (т. е. первые 3 в данном примере) и увеличиваем их размер на 1.
            # fold_sizes = [3, 3, 3, 2]
            fold_sizes[i] += 1  
        
        # Разделяем индексы на фолды
        np.random.shuffle(group_indices)
        current = 0
        group_folds = []
        for fold_size in fold_sizes:
            group_folds.append(group_indices[current:current + fold_size])
            current += fold_size
        
        # Заполняем итоговые списки train и test
        for i in range(k):
            test_indices_list[i].extend(group_folds[i])
            
            # Собираем train индексы, исключая текущий fold
            train_indices = []
            for j, fold in enumerate(group_folds):
                if j != i:  # Пропускаем текущий фолд
                    train_indices.extend(fold)
            train_indices_list[i].extend(train_indices)
    
    return [(train, test) for train, test in zip(train_indices_list, test_indices_list)]

In [16]:
# My 

my_fold_indices_skf = []
indices_kfs_my = k_fold_stratified(data, 'Pclass', 4)
for train, test in indices_kfs_my:
    print(len(train), len(test))
    my_fold_indices_skf.append((train, test))

668 223
668 223
668 223
669 222


In [17]:
n_splits = 4

for i in range(n_splits):
    print("распределение Pclass для train", list(data.iloc[my_fold_indices_skf[i][0]]['Pclass'].value_counts()),
          "распределение Pclass для test", list(data.iloc[my_fold_indices_skf[i][1]]['Pclass'].value_counts()))
    print("общее количество train и test", len(data.iloc[my_fold_indices_skf[i][0]]),
          len(data.iloc[my_fold_indices_skf[i][1]]))
    print("\n")

распределение Pclass для train [368, 162, 138] распределение Pclass для test [123, 54, 46]
общее количество train и test 668 223


распределение Pclass для train [368, 162, 138] распределение Pclass для test [123, 54, 46]
общее количество train и test 668 223


распределение Pclass для train [368, 162, 138] распределение Pclass для test [123, 54, 46]
общее количество train и test 668 223


распределение Pclass для train [369, 162, 138] распределение Pclass для test [122, 54, 46]
общее количество train и test 669 222




In [18]:
# n_splits>=2
from sklearn.model_selection import StratifiedKFold

fold_indices_skf = []
n_splits=4

skf = StratifiedKFold(n_splits) 

for train_index, test_index in skf.split(data.drop(['Pclass'], axis=1), data['Pclass']):
    
    fold_indices_skf.append([train_index,test_index])
    print(len(train_index), len(test_index))

668 223
668 223
668 223
669 222


In [19]:
for i in range(n_splits):
    print(list(data.iloc[fold_indices_skf[i][0]]['Pclass'].value_counts()),
          list(data.iloc[fold_indices_skf[i][1]]['Pclass'].value_counts()),
          len(data.iloc[fold_indices_skf[i][0]]),
          len(data.iloc[fold_indices_skf[i][1]]),
         )

[368, 162, 138] [123, 54, 46] 668 223
[368, 162, 138] [123, 54, 46] 668 223
[368, 162, 138] [123, 54, 46] 668 223
[369, 162, 138] [122, 54, 46] 669 222


### 4. K fold time series

**входные параметры:**  

data: выборка в виде pandas DataFrame, содержащая данные для разбиения.

n_splits: количество фолдов для кросс-валидации. Важно, что для временных рядов данные разбиваются по порядку, а не случайным образом.

**выходные данные:** 

Кортеж из двух списков:

Список индексов обучающей выборки для каждого фолда.

Список индексов тестовой выборки для каждого фолда.

**Описание:**

1) Делит данные на n_splits фолдов с использованием принципа временных рядов: каждый фолд содержит данные, которые идут последовательно во времени, и тестовые выборки всегда идут после обучающих.

Для каждого фолда данные разбиваются таким образом, что обучающая выборка всегда содержит все данные до определенной точки, а тестовая выборка — все данные, следующие за этой точкой.

Возвращает индексы строк обучающих и тестовых выборок для каждого фолда.  


**Пример:**  

Допустим у нас каждая строка это каждый день и мы делим выборку на 3 фолда 
тогда каждый fold будет выглядеть примерно так:  

**1 Fold:**  
**train:** 1 января по 31 марта  
**test:** 1 апреля по 30 июня      

**2 Fold:**  
**train:** 1 января по 30 июня  
**test:** 1 июля 30 сентября   

**3 Fold:**  
**train:** 1 января по 30 сентября  
**test:** 1 октября по 31 декабря   

In [20]:
def k_fold_time_series(data,k):
    
    random.seed(42)
    
    train_indices_list = []
    test_indices_list = []
    
    size = len(data)
    
    if k>size:
        k=size
    if k<1:
        k=1
    if isinstance(k,int)==False:
        k=int(k)
    
    # Генерируем случайные индексы и перемешиваем их
    indices = list(range(size))
    random.shuffle(indices)
    
    data = data.iloc[indices]
    
    size = len(data)
    step = size // (k + 1)  # Размер фолда
    
    for i in range(1, k + 1):
        train_indices = list(range(0, i * step))  # Все строки до текущего фолда
        test_indices = list(range(i * step, (i + 1) * step))  # Текущий фолд
        train_indices_list.append(train_indices)
        test_indices_list.append(test_indices)
    
    return list(zip(train_indices_list, test_indices_list))

**Сами создадим столбец с датой (time). Это нужно только для отслеживания динамики, сам этот столбец не используется в разбиении**

In [21]:
# Генерация столбца времени, начиная с 1 января 2000 года с интервалом 1 час
data['time'] = pd.date_range(start='2000-01-01', periods=len(data), freq='h')

In [22]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,time
0,0,3,0.188908,22.0,1,0,7.2500,0.339009,0.552795,0,2000-01-01 00:00:00
1,1,1,0.742038,38.0,1,0,71.2833,0.553571,0.552795,0,2000-01-01 01:00:00
2,1,3,0.742038,26.0,0,0,7.9250,0.339009,0.303538,1,2000-01-01 02:00:00
3,1,1,0.742038,35.0,1,0,53.1000,0.339009,0.552795,0,2000-01-01 03:00:00
4,0,3,0.188908,35.0,0,0,8.0500,0.339009,0.303538,1,2000-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0.188908,27.0,0,0,13.0000,0.339009,0.303538,1,2000-02-06 22:00:00
887,1,1,0.742038,19.0,0,0,30.0000,0.339009,0.303538,1,2000-02-06 23:00:00
888,0,3,0.742038,28.0,1,2,23.4500,0.339009,0.625774,0,2000-02-07 00:00:00
889,1,1,0.188908,26.0,0,0,30.0000,0.553571,0.303538,1,2000-02-07 01:00:00


In [23]:
# My 

my_fold_indices_tss = []
indices_kfs_my = k_fold_time_series(data, 3)
for train, test in indices_kfs_my:
    print(len(train), len(test))
    my_fold_indices_tss.append((train, test))

222 222
444 222
666 222


In [24]:
# Пройдем по всем фолдам
for i, (train_indices, test_indices) in enumerate(my_fold_indices_tss):
    # Получаем минимальное и максимальное время для обучающей выборки
    min_time_train = data.iloc[train_indices]['time'].min()
    max_time_train = data.iloc[train_indices]['time'].max()
    
    # Получаем минимальное и максимальное время для тестовой выборки
    min_time_test = data.iloc[test_indices]['time'].min()
    max_time_test = data.iloc[test_indices]['time'].max()
    
    # Выводим результаты для текущего фолда
    print(f"Fold {i + 1}:")
    print(f"  Min time (train): {min_time_train}, Max time (train): {max_time_train}")
    print(f"  Min time (test): {min_time_test}, Max time (test): {max_time_test}")
    print("-" * 50)

Fold 1:
  Min time (train): 2000-01-01 00:00:00, Max time (train): 2000-01-10 05:00:00
  Min time (test): 2000-01-10 06:00:00, Max time (test): 2000-01-19 11:00:00
--------------------------------------------------
Fold 2:
  Min time (train): 2000-01-01 00:00:00, Max time (train): 2000-01-19 11:00:00
  Min time (test): 2000-01-19 12:00:00, Max time (test): 2000-01-28 17:00:00
--------------------------------------------------
Fold 3:
  Min time (train): 2000-01-01 00:00:00, Max time (train): 2000-01-28 17:00:00
  Min time (test): 2000-01-28 18:00:00, Max time (test): 2000-02-06 23:00:00
--------------------------------------------------


In [25]:
# TimeSeriesSplit не смотрит на конкретный столбец, а работает только с порядком строк.

from sklearn.model_selection import TimeSeriesSplit

fold_indices_tss = []
tscv = TimeSeriesSplit(n_splits=3)

# Передаём данные (или их индексы) в метод split
for train, test in tscv.split(data):
    print(len(train),len(test))
    fold_indices_tss.append((train, test))

225 222
447 222
669 222


In [26]:
# Пройдем по всем фолдам
for i, (train_indices, test_indices) in enumerate(fold_indices_tss):
    # Получаем минимальное и максимальное время для обучающей выборки
    min_time_train = data.iloc[train_indices]['time'].min()
    max_time_train = data.iloc[train_indices]['time'].max()
    
    # Получаем минимальное и максимальное время для тестовой выборки
    min_time_test = data.iloc[test_indices]['time'].min()
    max_time_test = data.iloc[test_indices]['time'].max()
    
    # Выводим результаты для текущего фолда
    print(f"Fold {i + 1}:")
    print(f"  Min time (train): {min_time_train}, Max time (train): {max_time_train}")
    print(f"  Min time (test): {min_time_test}, Max time (test): {max_time_test}")
    print("-" * 50)

Fold 1:
  Min time (train): 2000-01-01 00:00:00, Max time (train): 2000-01-10 08:00:00
  Min time (test): 2000-01-10 09:00:00, Max time (test): 2000-01-19 14:00:00
--------------------------------------------------
Fold 2:
  Min time (train): 2000-01-01 00:00:00, Max time (train): 2000-01-19 14:00:00
  Min time (test): 2000-01-19 15:00:00, Max time (test): 2000-01-28 20:00:00
--------------------------------------------------
Fold 3:
  Min time (train): 2000-01-01 00:00:00, Max time (train): 2000-01-28 20:00:00
  Min time (test): 2000-01-28 21:00:00, Max time (test): 2000-02-07 02:00:00
--------------------------------------------------
